# Test Med Image Parse Plugin

This Jupyter notebook demonstrate how to test the med_image_parse.py plugin. Running this notebook allows the code to be tested without deployment.

Note: This notebook has the following dependencies:
- The MedImageParse model needs to be deployed before running this notebook. See https://aka.ms/MedImageParseDocs for deployment instructions.
- A CT image need to be uploaded to the `patient-data` blob container under folder `patient_4/images`.
- Update `patient_4/images/manifest.json` with the new CT image.

In [ ]:
import os
import sys

CUR_DIR = os.path.abspath("")
ROOT_DIR = os.path.dirname(os.path.dirname(CUR_DIR))
SRC_DIR = os.path.join(ROOT_DIR, 'src')

sys.path.append(SRC_DIR)
print(f"Added {SRC_DIR} to sys.path")

In [ ]:
from config import setup_logging
from dotenv import load_dotenv

setup_logging()
load_dotenv(os.path.join(SRC_DIR, '.env'))

In [ ]:
from app import create_app_context


app_ctx = create_app_context()

In [ ]:
from config import load_agent_config
from data_models.chat_context import ChatContext
from data_models.plugin_configuration import PluginConfiguration
from scenarios.default.tools.med_image_parse import create_plugin
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion


agent_config = load_agent_config("default")
radiology_agent_config = [config for config in agent_config if config["name"] == "Radiology"][0]
url = radiology_agent_config["hls_model_endpoint"].get("med_image_parse")
patient_id = "patient_4"
filename = "ct_scan.png"

kernel = Kernel()
kernel.add_service(
    AzureChatCompletion(
        service_id="default",
        deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        api_version="2024-10-21"
    )
)

chat_ctx = ChatContext("test-conversation-id")
chat_ctx.patient_id = patient_id

plugin_config = PluginConfiguration(
    kernel=kernel,
    chat_ctx=chat_ctx,
    agent_config=radiology_agent_config,
    data_access=app_ctx.data_access,
    azureml_token_provider=app_ctx.azureml_token_provider)

med_image_parse_plugin = create_plugin(plugin_config)
await med_image_parse_plugin.calculate_tumor_size(patient_id, filename, "tumor")